In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from credentials import Credentials as cr
import matplotlib as plt
import os
from split import Splitter

splitter = Splitter()

## Création et import du jeu de données principal

In [39]:
valeurs2019 = pd.read_csv(cr.PATH+'2019.txt', sep='|', usecols=['Nature mutation', 'Code postal','Nombre pieces principales', 'Surface terrain', 'Surface Carrez du 1er lot', 'Surface Carrez du 2eme lot', 'Surface Carrez du 3eme lot', 'Surface Carrez du 4eme lot', 'Surface Carrez du 5eme lot', 'Valeur fonciere'])
valeurs2019.head()

valeurs2019.to_csv(cr.CURATED_LOCAL_PATH + 'filtered2019.csv', index=False)

## Split des natures de mutations pour n'utiliser que les ventes.

In [33]:
splitter.split_datas('filtered2019.csv', 'Nature mutation')

../data/CURATED/Vente.csv
../data/CURATED/Echange.csv
../data/CURATED/Vente terrain à bâtir.csv
../data/CURATED/Vente en l&#39;état futur d&#39;achèvement.csv
../data/CURATED/Adjudication.csv
../data/CURATED/Expropriation.csv


In [12]:
ventes = pd.read_csv(cr.CURATED_LOCAL_PATH + "Vente.csv")
ventes = ventes.drop(columns=['Unnamed: 0'])

# Vérification du ratio de valeurs nulles en vue de nettoyer les données.
nancount = ventes["Surface terrain"].isna()
print(nancount.sum(), ' valeurs nulles / ', ventes["Surface terrain"].count(), ' valeurs totales.')


610423  valeurs nulles /  1698116  valeurs totales.


In [9]:
ventes.head()

,Nature mutation,Valeur fonciere,Code postal,Surface Carrez du 1er lot,Surface Carrez du 2eme lot,Surface Carrez du 3eme lot,Surface Carrez du 4eme lot,Surface Carrez du 5eme lot,Nombre pieces principales,Surface terrain
0,Vente,"37220,00",1000.0,"19,27",NaN,NaN,NaN,NaN,1.0,NaN
1,Vente,"185100,00",1000.0,NaN,"61,51",NaN,NaN,NaN,3.0,NaN
2,Vente,"185100,00",1000.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,Vente,"209000,00",1160.0,NaN,NaN,NaN,NaN,NaN,4.0,940.0
4,Vente,"134900,00",1370.0,NaN,NaN,NaN,NaN,NaN,5.0,490.0


In [10]:
nancount = ventes["Valeur fonciere"].isna()
print (nancount.sum())

27270


In [14]:
# ventes = ventes.dropna()
# ventes.head()

27270
